# MON NAIVE BAYES

In [1]:
%pip install spacy nltk
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\CJ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 2.2 MB/s eta 0:00:06
      --------------------------------------- 0.3/12.8 MB 3.2 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/12.8 MB 4.9 MB/s eta 0:00:03
     --- ------------------------------------ 1.1/12.8 MB 5.9 MB/s eta 0:00:02
     ---- ----------------------------------- 1.3/12.8 MB 5.6 MB/s eta 0:00:03
     ---- ----------------------------------- 1.6/12.8 MB 5.9 MB/s eta 0:00:02
     ----- ---------------------------------- 1.8/12.8 MB 5.5 MB/s eta 0:00:02
     ------ --------------------------------- 2.1/12.8 MB 5.6 MB/s eta 0:00:02
     ------- -------------------------------- 2.3/12.8 MB 5.3 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 5.5 MB/s eta 0:00:02
     --------- ------------------------------ 2.9/12.8 MB 5.6 MB/s eta 0:00:02
     ---------- ----------------------------- 3.3/12.8 MB 5


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\CJ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, words
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import spacy
import string
from spacy.lang.en import English
from spacy.tokens import Doc
from spacy.tokenizer import Tokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem import LancasterStemmer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CJ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CJ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\CJ\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
df = pd.read_csv('./tweet_emotions.csv')
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [4]:
# Initialize the Lancaster Stemmer from NLTK
stemmer = PorterStemmer()

# Set of valid words from NLTK's words corpus
# valid_words = set(words.words())

# Function to clean text, perform stemming, and filter words not in the corpus
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Replace non-alphanumeric characters with a single space
    text = re.sub(r'[^a-zA-Z0-9@]', ' ', text)
    
    # remove @ mention 
    # Split the text into words
    words = text.split()

    # Filter out words that start with '@'
    filtered_words = [word for word in words if not word.startswith('@')]

    # Join the filtered words back into a string
    text = ' '.join(filtered_words)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize the text
    words = nltk.word_tokenize(text)

    # Filter words not in the corpus
    # cleaned_words = [word for word in words if word in valid_words]

    # Remove stopwords and perform stemming
    stemmed_words = [stemmer.stem(word) for word in words if word not in stopwords.words('english') and '@' not in word]
    
    cleaned_text = ' '.join(stemmed_words)
    
    return cleaned_text
# Apply the cleaning function to the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)

# Display the cleaned dataset
print(df[['content', 'cleaned_content']].head())

                                             content  \
0  @tiffanylue i know  i was listenin to bad habi...   
1  Layin n bed with a headache  ughhhh...waitin o...   
2                Funeral ceremony...gloomy friday...   
3               wants to hang out with friends SOON!   
4  @dannycastillo We want to trade with someone w...   

                                     cleaned_content  
0  know listenin bad habit earlier start freakin ...  
1             layin n bed headach ughhhh waitin call  
2                       funer ceremoni gloomi friday  
3                              want hang friend soon  
4               want trade someon houston ticket one  


Data Preparation

In [46]:
# Hapus data yg kosong
df.dropna(inplace = True)

# Hapus data duplikat
df.drop_duplicates(inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39173 entries, 1 to 39999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tweet_id         39173 non-null  int64 
 1   sentiment        39173 non-null  object
 2   content          39173 non-null  object
 3   cleaned_content  39173 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [5]:
for x in df.index:
  if df.loc[x, "sentiment"] == 'empty':
    df.drop(x,inplace=True)

In [6]:
df.head()

,tweet_id,sentiment,content,cleaned_content
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...,layin n bed headach ughhhh waitin call
2,1956967696,sadness,Funeral ceremony...gloomy friday...,funer ceremoni gloomi friday
3,1956967789,enthusiasm,wants to hang out with friends SOON!,want hang friend soon
4,1956968416,neutral,@dannycastillo We want to trade with someone w...,want trade someon houston ticket one
5,1956968477,worry,Re-pinging @ghostridah14: why didn't you go to...,ping go prom bc bf like friend


In [7]:
%pip install BeautifulSoup4 scikit-learn textblob 
import requests
from bs4 import BeautifulSoup
import pandas as pd
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import metrics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\CJ\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Multinomial NB

In [16]:
from sklearn.model_selection import train_test_split

# Load the training data from the CSV file
df_train_multinomial = df

# Preprocess the training data
x = df_train_multinomial['cleaned_content']
y = df_train_multinomial['sentiment']

# using count vectorizer
vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(x)
# print(x_train_vectorized)
# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x_train_vectorized, y, test_size=0.12, random_state=42)

# initialize multinomialNB classifier
classifier = MultinomialNB()
classifier.fit(X_train.toarray(), y_train)

# Predict on the test data
y_pred = classifier.predict(X_test.toarray())
print(y_pred)
print(y_test)
# Evaluate the accuracy of the classifier
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


['relief' 'happiness' 'happiness' ... 'sadness' 'sadness' 'worry']
13096       relief
26315      neutral
27891    happiness
14035        worry
37203        worry
           ...    
11403    happiness
14837      neutral
12283         hate
10098         hate
11414      sadness
Name: sentiment, Length: 4701, dtype: object
Accuracy: 0.32929164007657946


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Load the training data from the CSV file
df_train_gaussian = df

# Preprocess the training data
x = df_train_gaussian['cleaned_content']
y = df_train_gaussian['sentiment']

# using count vectorizer
vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(x)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(x_train_vectorized, y, test_size=0.2, random_state=42)

# Initialize the Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Train the classifier on the training data
gnb.fit(X_train.toarray(), y_train)

# Predict on the test data
y_pred = gnb.predict(X_test.toarray())

# Evaluate the accuracy of the classifier
accuracy = metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.09649010848755583


In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

titles = ["magics-return-i-can-see-the-spirits", "taking-the-mafia-to-the-magic-world", "ill-surpass-the-mc"]

for title in titles:
    url = f'https://www.lightnovelworld.com/novel/{title}'
    response = requests.get(url, headers=headers)

    data_dict = {'title': '', 'author': '', 'rating': '', 'categories': [], 'comments': [], 'sentiments': []}

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Get novel title
        title_element = soup.find('h1', class_='novel-title')
        if title_element:
            data_dict['title'] = title_element.text.strip()

        # Get author information
        author_element = soup.find('span', attrs={'itemprop': 'author'})
        if author_element:
            data_dict['author'] = author_element.text.strip()

        # Get rating information
        meta_rating_element = soup.find('meta', attrs={'itemprop': 'ratingValue'})
        if meta_rating_element:
            data_dict['rating'] = meta_rating_element.get('content', '').strip()

        # Mencari semua elemen <p> di dalam <div> dengan class 'comment-text'
        comment_text_divs = soup.find_all('div', class_='comment-text')

        for comment_text_div in comment_text_divs:
            p_elements = comment_text_div.find_all('p')

            for p_element in p_elements:
                paragraph_text = p_element.text.strip()
                data_dict['comments'].append(paragraph_text)

                # Perform sentiment analysis using TextBlob
                blob = TextBlob(paragraph_text)
                sentiment = 'positive' if blob.sentiment.polarity > 0 else 'negative' if blob.sentiment.polarity < 0 else 'neutral'
                data_dict['sentiments'].append(sentiment)

        # Get Categories
        categories_header = soup.find('h4', string='Categories')
        if categories_header:
            categories_list = []
            ul_element = categories_header.find_next('ul')
            if ul_element:
                categories = ul_element.find_all('li')
                for category in categories:
                    category_text = category.text.strip()
                    categories_list.append(category_text)
                data_dict['categories'] = categories_list

        # Print each comment with its predicted sentiment
        print(data_dict)
        for comment, sentiment in zip(data_dict['comments'], data_dict['sentiments']):
            print(f"Comment: {comment}")
            print(f"Predicted Sentiment: {sentiment}\n")
    else:
        print(f"Gagal mengambil halaman web untuk judul {title}. Kode status:", response.status_code)


{'title': "Magic's Return: I Can See The Spirits", 'author': 'SleepDeprivedSloth', 'rating': '4.0', 'categories': ['Fantasy', 'Action', 'Romance', 'Comedy'], 'comments': ['Any novels with similar romance? Pls reccomend me some...', 'The novel has good romance, pacing, fight scenes, but Vaan feels like a 15 year old.\nWhat 31 year old genocidal mob boss says things like “simp general” and has the patience and ego of a toddler? His *ss would’ve gotten killed years ago.\nIf you can get past the fact the mc is basically an egotistical teenager with no character development yet, the rest of the novel is enjoyable.', 'I can’t get over his personality. He literally attacks people and when they defend themselves he says “my life is so precious how dare he try to take it from me”', 'Well I would argue that isn’t actually as bad as some other actions he takes as most gangsters use self-justification as a way to cope with the crimes they commit. I personally can’t get over how he literally acts l